# 1. Data Loading and Basis Info
* Data loaded to df from '/kaggle/input/income-urban-vs-rural-for-each-county/Income_Urban_VS_Rural.csv'
* 3,222 non-null records as expected from dataset notes

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/income-urban-vs-rural-for-each-county/Income_Urban_VS_Rural.csv


In [2]:
# Remove pesky warning messages + import graphing libraries
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv('/kaggle/input/income-urban-vs-rural-for-each-county/Income_Urban_VS_Rural.csv')
print(data.info()) #No Nulls Found
print(data.head(5)) #Quick print of head of df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3222 entries, 0 to 3221
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   County                   3222 non-null   object
 1   State                    3222 non-null   object
 2   FIPS                     3222 non-null   int64 
 3   State FIPS Code          3222 non-null   int64 
 4   County FIPS Code         3222 non-null   int64 
 5   Total Population         3222 non-null   int64 
 6   Median Household Income  3222 non-null   int64 
 7   Urban-Rural              3222 non-null   object
dtypes: int64(5), object(3)
memory usage: 201.5+ KB
None
           County    State  FIPS  State FIPS Code  County FIPS Code  \
0  Autauga County  Alabama  1001                1                 1   
1  Baldwin County  Alabama  1003                1                 3   
2  Barbour County  Alabama  1005                1                 5   
3     Bibb County  Alab

# 2. Data Groupings to locate interesting targets

In [4]:
#Code to help the text display without truncation / wrapping
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

#Group 1: Top 5 states by population (sorted)
Group1_df = data.groupby('State').agg({'Total Population': 'sum'}).sort_values(by='Total Population', ascending=False)
Group1_df['Total Population'] = Group1_df['Total Population'].apply(lambda x: f'{x:,}')

print('The Top 5 States by Population are as follows: \n')
print(Group1_df.head(5))
print('\n\n')

#Group 2: Top 5 states by maximum median household income (sorted)
Group2_df = data.groupby('State').agg({'Median Household Income': 'max'}).sort_values(by='Median Household Income', ascending=False)
Group2_df =Group2_df.rename(columns={'Median Household Income' : 'Max Median Household Income'})
Group2_df['Max Median Household Income'] = Group2_df['Max Median Household Income'].apply(lambda x: f'${x:,}')
print('The Top 5 States by Maximum Median Household Income are as follows: \n')
print(Group2_df.head(5))
print('\n\n')

#Group 3: Top 5 states by taxable personal income (e.g. median x population) (sorted)
Group3_df = data
Group3_df['Income Estimate'] = Group3_df['Total Population'] * Group3_df['Median Household Income']
Group3_df = Group3_df.groupby('State').agg({'Income Estimate' : 'sum'})
Group3_df['Income Estimate'] = (Group3_df['Income Estimate']/1000000000).round(2).apply(lambda x: f'${x:,} Billion')
print('The Top 5 States by Taxable Personal Income are as follows: \n')
print(Group3_df.head(5))

The Top 5 States by Population are as follows: 

             Total Population
State                        
California         39,242,785
Texas              29,640,343
Florida            21,928,881
New York           19,872,319
Pennsylvania       12,986,518



The Top 5 States by Maximum Median Household Income are as follows: 

           Max Median Household Income
State                                 
Virginia                      $178,707
California                    $159,674
Maryland                      $146,982
Colorado                      $145,737
New York                      $143,408



The Top 5 States by Taxable Personal Income are as follows: 

              Income Estimate
State                        
Alabama       $320.57 Billion
Alaska         $65.82 Billion
Arizona        $566.7 Billion
Arkansas      $181.82 Billion
California  $3,894.23 Billion


In [5]:
#Group 4: Top 2 counties per state by population (sorted)
Group4_df = data.groupby('State').apply(lambda x: x.nlargest(2, 'Total Population')).reset_index(drop=True)

state_sums_1 = Group4_df.groupby('State')['Total Population'].sum().sort_values(ascending=False) #generate order by max population to use for sorting output
Group4_df['State'] = pd.Categorical(Group4_df['State'], categories=state_sums_1.index, ordered=True)
Group4_df = Group4_df.sort_values(by='State')

Group4_df = Group4_df[['State', 'County', 'Total Population']]
Group4_df['Total Population'] = Group4_df['Total Population'].apply(lambda x: f'{x:,}')

print('The Top 2 Counties by Population from the Top 10 States are as follows: \n')
print(Group4_df.head(20).to_string(index=False)) #hiding the index rather than going through another reset just for printing
print('\n\n')


#Group 5: Top 2 counties per state by median income (sorted)
Group5_df = data.groupby('State').apply(lambda x: x.nlargest(2, 'Median Household Income')).reset_index(drop=True)

state_sums_2 = Group5_df.groupby('State')['Median Household Income'].sum().sort_values(ascending=False) #generate order by max Median Household Income to use for sorting output
Group5_df['State'] = pd.Categorical(Group5_df['State'], categories = state_sums_2.index, ordered=True)
Group5_df = Group5_df.sort_values(by='State')

Group5_df = Group5_df[['State', 'County', 'Median Household Income']]
Group5_df['Median Household Income'] = Group5_df['Median Household Income'].apply(lambda x: f'${x:,}')
print('The Top 2 Counties by Median Household Income from the Top 10 States are as follows: \n')
print(Group5_df.head(20).to_string(index=False))
print('\n\n')

The Top 2 Counties by Population from the Top 10 States are as follows: 

       State              County Total Population
  California  Los Angeles County        9,848,406
  California    San Diego County        3,282,782
       Texas       Harris County        4,758,579
       Texas       Dallas County        2,603,816
    Illinois       DuPage County          927,263
    Illinois         Cook County        5,185,812
     Arizona     Maricopa County        4,491,987
     Arizona         Pima County        1,049,947
    New York        Kings County        2,646,306
    New York       Queens County        2,330,124
     Florida      Broward County        1,946,127
     Florida   Miami-Dade County        2,685,296
  Washington       Pierce County          924,106
  Washington         King County        2,262,713
    Michigan        Wayne County        1,773,767
    Michigan      Oakland County        1,272,294
Pennsylvania Philadelphia County        1,582,432
Pennsylvania    Allegheny 